#### 📦 Importing Required Libraries
We import PyTorch, TorchVision, scikit-learn, and other utilities.


In [ ]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

#### 🔧 Setting Device and Random Seed
This ensures reproducibility and leverages GPU acceleration if available.


In [3]:
# Set random seed
torch.manual_seed(42)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


#### 🔄 Data Preprocessing & Augmentation
We define transformations for both training and validation datasets.


In [5]:
# Data Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

#### 📂 Loading Dataset
Dataset is structured with `train/` and `val/` directories, each containing `male/` and `female/` subfolders.


In [6]:
# Data directory
data_dir = 'Task_A'

In [7]:
# Datasets and loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(f"Dataset sizes: {dataset_sizes}")

Dataset sizes: {'train': 1926, 'val': 422}


#### 🧠 Model Architecture: Custom ResNet50
We use a pre-trained ResNet50, freeze 1st layer, and add a custom classifier head for binary classification.


In [8]:
# Custom ResNet50 model
class CustomResNet50(nn.Module):
    def __init__(self):
        super(CustomResNet50, self).__init__()
        base_model = models.resnet50(pretrained=True)

        # Unfreeze layer2 and layer3 and layer4
        for name, param in base_model.named_parameters():
            if 'layer2' in name or 'layer3' in name or 'layer4' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)  # binary classification
        )

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [9]:
resnet50 = CustomResNet50().to(device)

#### ⚙️ Define Loss Function, Optimizer, and Scheduler
We use CrossEntropy with label smoothing and apply weight decay regularization.


In [10]:
# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet50.parameters()), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

#### 🏋️‍♂️ Model Training Loop
Training and validation loops are implemented with metrics logged for each epoch.


In [11]:
# Training loop
num_epochs = 30
best_acc = 0.0
best_model_wts = copy.deepcopy(resnet50.state_dict())

In [12]:
train_acc_list, val_acc_list = [], []
train_loss_list, val_loss_list = [], []

In [13]:
for epoch in range(num_epochs):
    print(f'\nEpoch {epoch + 1}/{num_epochs}')
    print('-' * 20)

    for phase in ['train', 'val']:
        if phase == 'train':
            resnet50.train()
        else:
            resnet50.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = resnet50(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        if phase == 'train':
            train_loss_list.append(epoch_loss)
            train_acc_list.append(epoch_acc.item())
            scheduler.step()
        else:
            val_loss_list.append(epoch_loss)
            val_acc_list.append(epoch_acc.item())
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(resnet50.state_dict())

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print(f"\nTraining complete. Best val Acc: {best_acc:.4f}")


Epoch 1/30
--------------------
train Loss: 0.3954 Acc: 0.8738
val Loss: 0.3581 Acc: 0.8981

Epoch 2/30
--------------------
train Loss: 0.3436 Acc: 0.9133
val Loss: 0.3003 Acc: 0.9455

Epoch 3/30
--------------------
train Loss: 0.3203 Acc: 0.9315
val Loss: 0.3217 Acc: 0.9313

Epoch 4/30
--------------------
train Loss: 0.3177 Acc: 0.9367
val Loss: 0.3262 Acc: 0.9360

Epoch 5/30
--------------------
train Loss: 0.3089 Acc: 0.9330
val Loss: 0.3100 Acc: 0.9408

Epoch 6/30
--------------------
train Loss: 0.3005 Acc: 0.9460
val Loss: 0.3400 Acc: 0.9194

Epoch 7/30
--------------------
train Loss: 0.2979 Acc: 0.9408
val Loss: 0.3582 Acc: 0.8981

Epoch 8/30
--------------------
train Loss: 0.2875 Acc: 0.9470
val Loss: 0.3917 Acc: 0.9028

Epoch 9/30
--------------------
train Loss: 0.2899 Acc: 0.9470
val Loss: 0.3320 Acc: 0.9313

Epoch 10/30
--------------------
train Loss: 0.2923 Acc: 0.9491
val Loss: 0.3345 Acc: 0.9194

Epoch 11/30
--------------------
train Loss: 0.2772 Acc: 0.9559
val 

In [ ]:
resnet50.load_state_dict(best_model_wts)  # ✅ Restore best weights
torch.save(resnet50.state_dict(), 'Gender_Classification_model.pt')  # Save the  model

#### 📈 Evaluation on Training and Validation Sets
Metrics include Accuracy, Precision, Recall, and F1-Score.


In [ ]:
# ===== Evaluate Best Saved Model on Validation and Training Sets =====
print("\n🔍 Evaluating best saved model on training and validation sets...")

# Load the best model weights
resnet50.load_state_dict(torch.load('Gender_Classification_model.pt'))
resnet50.eval()

def evaluate_model(dataloader, dataset_name="Dataset"):
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = resnet50(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"\n📊 Evaluation on {dataset_name}:")
    print(f"  - Accuracy : {acc:.4f}")
    print(f"  - Precision: {prec:.4f}")
    print(f"  - Recall   : {rec:.4f}")
    print(f"  - F1-Score : {f1:.4f}")

# Evaluate on training set
evaluate_model(dataloaders['train'], dataset_name="Training Set")

# Evaluate on validation set
evaluate_model(dataloaders['val'], dataset_name="Validation Set")



🔍 Evaluating best saved model on training and validation sets...

📊 Evaluation on Training Set:
  - Accuracy : 0.9346
  - Precision: 0.9426
  - Recall   : 0.9821
  - F1-Score : 0.9620

📊 Evaluation on Validation Set:
  - Accuracy : 0.9455
  - Precision: 0.9651
  - Recall   : 0.9679
  - F1-Score : 0.9665


#### 🧪 Test Script (test_Gender_Classification_model.py)
This standalone script accepts test data directory and model path and prints evaluation metrics. It uses the same model architecture and transforms as in training.


In [1]:
%%writefile test_Gender_Classification_model.py
import argparse
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

class CustomResNet50(nn.Module):
    def __init__(self):
        super(CustomResNet50, self).__init__()
        base_model = models.resnet50(pretrained=True)
        for name, param in base_model.named_parameters():
            if 'layer2' in name or 'layer3' in name or 'layer4' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        self.classifier = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"📊 Evaluation Metrics on Test Set:")
    print(f"  - Accuracy : {acc:.4f}")
    print(f"  - Precision: {prec:.4f}")
    print(f"  - Recall   : {rec:.4f}")
    print(f"  - F1-Score : {f1:.4f}")

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, required=True, help='Path to test data directory')
    parser.add_argument('--weights', type=str, default='Gender_Classification_model.pt', help='Path to saved model weights')
    args = parser.parse_args()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    test_dataset = datasets.ImageFolder(args.data_dir, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

    model = CustomResNet50().to(device)
    model.load_state_dict(torch.load(args.weights, map_location=device))

    evaluate(model, test_loader, device)

if __name__ == '__main__':
    main()

Writing test_Gender_Classification_model.py
